<a href="https://colab.research.google.com/github/MK316/workshops/blob/main/20240531_hufs/240531_HUFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌌 **Digital Transformation in Classroom: + Coding**

+ Date: May 31, 2024
+ Hosted by: Graduate School of Education at HUFS
+ Online workshop
+ Speaker: Miran Kim (Associate Professor, Dept. of English Education at GNU)

# 🌿 **Part 3. Coding to the classroom**

## **I. Introduction**

+ Understanding Digital Transformation in Education
+ Introduction to Python Coding

##📌 Setting: Install necessary Python packages

In [ ]:
#@markdown 🌱 Set-up: install, import libraries to use {pyqrcode, gtts, IPython, pandas, numpy, nltk, random}
%%capture
!pip install pyqrcode gtts wordcloud seaborn gradio
!wget https://github.com/google/fonts/raw/main/ofl/nanumgothic/NanumGothic-Regular.ttf -O NanumGothic.ttf
!pip install gspread gspread-dataframe

# display, qr code
from IPython.display import YouTubeVideo, Image, Audio, display
!pip install gtts
import pandas as pd
import pyqrcode
from pyqrcode import QRCode
from gtts import gTTS
from ipywidgets import widgets
import os
import numpy as np
import random
# wordcloud
!pip install wordcloud nltk
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
english_stopwords = set(stopwords.words('english'))

import urllib.request

## 🌀 **Activity: On-site Survey**

+ [설문링크](https://forms.gle/7NYj7QEVwWCs4pbr5)

In [ ]:
#@markdown 📢 Instruction: (in Korean)
def tts(text):
  tts = gTTS(text, lang = "ko", slow = False)
  tts.save("myaudio.mp3")
  return Audio("myaudio.mp3")

txt = """
안녕하세요 여러분? 시작에 앞서 아주 간단한 설문을 함께 해 보도록 하겠습니다. 설문은 간단한 4개의 문항이며 30초 정도 소요될 것입니다.
본인 스마트폰 카메라 앱을 열어 아래 큐알코드를 읽은 후, 설문 링크를 따라가서 설문을 마쳐 주시기 바랍니다. 결과는 한 2분 후쯤? 김미란 선생님이 설명해 주실 거예요.
"""

tts(txt)
print("Instruction (in Korean)")
display(Audio("myaudio.mp3", autoplay=False))
print("="*30)
print("Instruction (in English)")
url = "https://github.com/MK316/workshops/raw/main/240531HUFS/0531_survey.wav"
Audio(url)

In [ ]:
#@markdown Survey link QR code:
s = "https://forms.gle/3jrJYQZYtYsK5TeQA"

# Generate QR code
url = pyqrcode.create(s)

# Create and save the png file naming "myqr.png"
url.svg("myqrcode.svg", scale=12)

from IPython.display import SVG, display
def show_svg(file):
    display(SVG(file))

show_svg("myqrcode.svg")

## 🌀 **Survey Data analysis using coding**

+ Using online surveys and analyzing the results immediately can greatly benefit students in the classroom by providing:
  + instant feedback,
  + identifying learning gaps, and
  + enabling tailored instruction to enhance understanding and engagement.

In [ ]:
#@markdown **Data to read**
# !pip install gspread gspread-dataframe

# Authenticate and connect to Google Drive
from google.colab import auth
auth.authenticate_user()

# Authorize and initialize the gspread client
import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheet
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1YAm57huSjQs7VwRtWrDcMf2G_0i8tG36yNIfZJSH3Fg/edit?usp=sharing'
sheet = gc.open_by_url(spreadsheet_url).sheet1  # Open the first sheet


# Convert to pandas DataFrame
import pandas as pd
from gspread_dataframe import get_as_dataframe

# Ensure the header is correctly detected
data = sheet.get_all_values()
df = pd.DataFrame(data)
df.columns = df.iloc[0]  # Set the first row as the header
df = df[1:]  # Remove the header row from the data

# Remove 'Unnamed' columns
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Rename specific columns
df.columns.values[2] = "Q2"
df.columns.values[3] = "Q4"
df.columns.values[4] = "Q1"
df.columns.values[5] = "Q3"

# Rearrange the columns
df = df[['Q1', 'Q2', 'Q3', 'Q4'] + [col for col in df.columns if col not in ['Q1', 'Q2', 'Q3', 'Q4']]]

df['Q2'] = pd.to_numeric(df['Q2'], errors='coerce')
df = df.dropna(subset=['Q2'])


# Remove rows with 'NaN' values
df = df.dropna(how='any')
df = df.drop(columns=["E-mail"])

# Display the cleaned DataFrame with new column names
print("Number of respondence:", len(df['Q1']))
print("="*80)
df

In [ ]:
#@markdown **Random pick among 'Q1'**

import numpy as np
from gtts import gTTS
from IPython.display import Audio

# Assuming df is your DataFrame and 'Q1' column contains 4-digit strings

# Pick a random value from the 'Q1' column
random_value = np.random.choice(df['Q1'])
print(f"Randomly selected value from Q1: {random_value}")

# Generate the text to read each digit individually
digits_text = ' '.join(random_value)

# Generate audio calling out the number
tts = gTTS(text=f"The selected number is... {digits_text}!", lang='en')
tts.save("selected_number.mp3")

# Play the audio
Audio("selected_number.mp3", autoplay=True)


In [ ]:
#@markdown **Q2 analysis (AI digital literacy perception): Generate a boxplot**
import seaborn as sns
import matplotlib.pyplot as plt


mean_value = df['Q2'].mean()

print(f"Mean of Q2: {mean_value}")

plt.figure(figsize=(10, 6))
sns.boxplot(y=df['Q2'])
plt.title('Boxplot of Q2. AI Digital Listeracy (self-evaluated)')
plt.ylabel('Q2 Values')
plt.ylim(0,6)
plt.show()

In [ ]:
#@markdown **Q3 analysis (Python Coding familiarity)**

# Define the order of the responses
response_order = [
    "Never used it.",
    "Heard of it but never used it.",
    "Tried it a few times.",
    "Use it occasionally.",
    "Use it regularly."
]

# Clean 'Q3' column to remove any extra spaces
df['Q3'] = df['Q3'].str.strip()

# Ensure 'Q3' column contains the responses in the defined order
df['Q3'] = pd.Categorical(df['Q3'], categories=response_order, ordered=True)

# Count the occurrences of each response in 'Q3' (Python Coding)
response_counts = df['Q3'].value_counts().reindex(response_order)

# Generate a barplot with gradient colors from yellow to green
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Create a gradient color palette
num_colors = len(response_counts)
colors = sns.color_palette("YlGn", num_colors)

plt.figure(figsize=(12, 8))
bars = plt.bar(response_counts.index, response_counts.values, color=colors)

# Add labels to each bar
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, int(yval), ha='center', va='bottom', fontsize=12)

plt.title('Responses for Q3')
plt.xlabel('Responses')
plt.ylabel('Counts')
plt.xticks(rotation=45)
plt.show()


In [ ]:
#@markdown **Q4 analysis: Keywords in wordcloud**

# Install required libraries
# !pip install gspread gspread-dataframe wordcloud

# Generate a word cloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt


# Combine words under 'Q2' column
text = ' '.join(df['Q4'])

wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


---
# II. **Python Coding for Teachers**

1. **Enhanced Interactivity:**

+ Customizable Learning Tools: Coding allows educators and students to create interactive tools such as quizzes, flashcards, and language games, tailored to specific learning needs.
+ Dynamic Content: APIs can provide access to up-to-date content, including news articles, language exercises, and multimedia resources.


2. **Personalized Learning:**

+ Adaptive Learning Platforms: Coding enables the development of adaptive learning platforms that adjust to individual students' proficiency levels, providing personalized feedback and recommendations.
+ Data Analytics: APIs can collect and analyze data on students' performance, helping educators to identify areas where students struggle and to tailor instruction accordingly.

3. **Access to Diverse Resources:**

+ Language Databases: APIs can connect to extensive language databases and dictionaries, providing instant access to vocabulary, translations, and pronunciation guides.
+ Cultural Immersion: Students can access authentic language resources such as videos, podcasts, and social media feeds from native speakers.

4. **Automation of Administrative Tasks:**

+ Grading and Assessment: Coding can automate the grading of language exercises and quizzes, saving educators time and providing immediate feedback to students.

---
### Samples

+ Language Application: Interactive, customized, student-centered, and **multi-modal applications** are possible.
+ Using Huggingface: It allows the application to run on an independent platform, enabling students to use the application outside of class.


### **📙 Simple Language Application [1]**

In [ ]:
#@markdown **Text-to-Speech (TTS) application: Korean, English (AmE), English (BrE), and French**

#!pip install gtts gradio (This needs installation to work alone.)

import gradio as gr
from gtts import gTTS

def text_to_speech(text, language):
    language_map = {
        "🇰🇷 Korean": "ko",
        "🇺🇸 English (AmE)": ("en", "us"),
        "🇬🇧 English (BrE)": ("en", "co.uk"),
        "🇫🇷 French": "fr",
        "🇪🇸 Spanish": ("es", "es"),
        "🇨🇳 Chinese": "zh-CN"
    }

    if isinstance(language_map[language], tuple):
        lang, tld = language_map[language]
        tts = gTTS(text=text, lang=lang, tld=tld)
    else:
        lang = language_map[language]
        tts = gTTS(text=text, lang=lang)

    tts.save("output.mp3")
    return "output.mp3"

# Define the Gradio interface
iface = gr.Interface(
    fn=text_to_speech,
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter text here..."),
        gr.Radio(["🇰🇷 Korean", "🇺🇸 English (AmE)", "🇬🇧 English (BrE)", "🇫🇷 French", "🇪🇸 Spanish", "🇨🇳 Chinese"], label="Language")
    ],
    outputs=gr.Audio(type="filepath"),
    title="Text to Speech Application (Multi-languages)",
    description="Enter text and choose a language to generate the corresponding audio."
)

# Launch the Gradio interface
iface.launch(debug=True)


In [ ]:
#@markdown QR code generation: paste your link
s = input()

# Generate QR code
url = pyqrcode.create(s)

# Create and save the png file naming "myqr.png"
url.svg("myqrcode.svg", scale=12)

from IPython.display import SVG, display
def show_svg(file):
    display(SVG(file))

show_svg("myqrcode.svg")

### **📙 Simple Language Application [2]: Language practice activity using NLP (Natural Language Processing)**

+ [Sample applications online](https://mrkim21.github.io)


# Using API

# III. Practical Application and Tools

+ Educational tools and platforms that support Python coding
+ A simple Natural Language Processing (NLP) project for text analysis
+ Interactive coding session

# IV. Challenges and Solutions

+ Common challenges faced by language teachers in learning and teaching coding
+ Strategies for overcoming these challenges
+ Sharing resources and support networks for continuous learning

# Q&As

# Suggested Resources

+ Online Python tutorials and courses (e.g., Codecademy, Coursera)
+ Python documentation and community forums (e.g., Python.org, Stack Overflow)
+ Educational tools (e.g., Jupyter Notebooks, Google Colab)
+ Books and articles on Python in education